In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

# Data Preprocessing for Matrix User x Item

In [2]:
#Source Code for data Processing
#https://gist.github.com/victorkohler/f48ea6512058719ba52053851fedc745

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
#np.shape(train_data)
np.shape(test_data)

(3782335, 12)

In [4]:
#train = train_data.append(test_data)
#np.shape(train)

In [5]:
item_metadata = pd.read_csv('item_metadata.csv', sep=',', engine='python')

In [6]:
train_v2 = train_data[train_data['action_type']=='clickout item']
train_v2= train_v2[train_v2.reference.notnull()]
train_v2.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
122,03K8AXBL4BX2,ec139e10b9238,1541100652,7,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...


In [7]:
train_v2.dtypes

user_id            object
session_id         object
timestamp           int64
step                int64
action_type        object
reference          object
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtype: object

In [8]:
train_v2["reference"]= train_v2["reference"].astype(int)
train_v2["step"]= train_v2["step"].astype(int)
train_v2.dtypes

user_id            object
session_id         object
timestamp           int64
step                int32
action_type        object
reference           int32
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtype: object

In [9]:
train_v2.head()
#np.shape(train_v2)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
122,03K8AXBL4BX2,ec139e10b9238,1541100652,7,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...


In [10]:
#train_v3 = train_v2.head(10000)
train_v3 = train_v2
train_v3.head()
np.shape(train_v3)

(1586586, 12)

In [11]:
data = train_v3[['user_id','reference']]
data.head()
#np.shape(data)

,user_id,reference
13,00RL8Z82B2Z1,109038
15,00RL8Z82B2Z1,1257342
115,02SRUT1NQYH1,2795374
121,03K8AXBL4BX2,1032816
122,03K8AXBL4BX2,1032816


In [12]:
data=data.groupby(['user_id','reference']).size().reset_index()
data.columns = ['user', 'item', 'click_count']
data.head()

,user,item,click_count
0,0001VQMGUI65,477811,1
1,0001VQMGUI65,950829,1
2,0001VQMGUI65,2019467,1
3,0001VQMGUI65,3133074,2
4,0003QTCX5MJX,2195060,1


In [13]:
user_id_table = data['user']
user_id_table = pd.DataFrame(user_id_table)
user_id_table = pd.DataFrame(user_id_table['user'].unique())
user_id_table.columns = ['user']
user_id_table['user_id'] = dict(enumerate(user_id_table['user'], start=0))
user_id_table.head()
#np.shape(user_id_table)

,user,user_id
0,0001VQMGUI65,0
1,0003QTCX5MJX,1
2,0004IOZI7CKF,2
3,0004WCFRV3FB,3
4,0006W0R5A5V8,4


In [14]:
item_id_table = data['item']
item_id_table = pd.DataFrame(item_id_table)
item_id_table = pd.DataFrame(item_id_table['item'].unique())
item_id_table.columns = ['item']
item_id_table['item_id'] = dict(enumerate(item_id_table['item'], start=0))
item_id_table.head()
#np.shape(item_id_table)

,item,item_id
0,477811,0
1,950829,1
2,2019467,2
3,3133074,3
4,2195060,4


In [15]:
data = data.merge(user_id_table, how='left', left_on='user', right_on = 'user').merge(item_id_table, how='left', left_on='item', right_on = 'item')
data.head()

,user,item,click_count,user_id,item_id
0,0001VQMGUI65,477811,1,0,0
1,0001VQMGUI65,950829,1,0,1
2,0001VQMGUI65,2019467,1,0,2
3,0001VQMGUI65,3133074,2,0,3
4,0003QTCX5MJX,2195060,1,1,4


# Graph Embedding

In [32]:
#Author Manuel Aguero
import scipy.sparse as sparse
import numpy as np
import random
import mpmath

def mini_batch_generator(graph,batch_size,row_length,col_length):
    i = random.randint(0, row_length-batch_size)
    j = random.randint(0,col_length-batch_size)
    return graph[i:i+batch_size,j:j+batch_size]

def negative_sampling(E,row_length,col_length,batch_size):
    S = sparse.dok_matrix((row_length+1,col_length+1)) #create empty negtive feedack graph
    for i in range(batch_size):
        a = random.randint(0, row_length)
        b = random.randint(0,col_length)
        randomEdge = (a,b) #randomly sample edges from the implicit feedback graph
        if (a,b) not in E:
            S[a,b]=1
    return S.tocoo()

def compute_e_ife(theta_u,theta_v,G_items,S_items):
    sumG = sumS = 0
    for (i,j) in G_items:
        sumG += np.log(1/(1+np.exp(np.dot(-theta_u[i],np.transpose(theta_v[j])))))
    for (i,j) in S_items:
        sumS += np.log(1 - (1/(1+np.exp(np.dot(-theta_u[i],np.transpose(theta_v[j]))))) )
    return sumG + sumS
            
def update_embeddings(theta_u,theta_v,G_mini_items,S_items,a,e_ife):
    sumGu = sumSu = sumGv = sumSv = 0
    for (i,j) in G_mini_items:
        currVal = (1 - 1/(1+np.exp(-np.dot(theta_u[i],np.transpose(theta_v[j])))))
        uVal = np.dot(currVal,theta_v[j])  
        vVal = np.dot(currVal,theta_u[i])
        sumGu += uVal
        sumGv += vVal
    for (i,j) in S_items:
        currVal = (-1/(1+np.exp(-np.dot(theta_u[i],np.transpose(theta_v[j])))))
        uVal = np.dot(currVal,theta_v[j])  
        vVal = np.dot(currVal,theta_u[i])
        sumSu += uVal
        sumSv += vVal
    theta_u = theta_u + (a*sumGu + a*sumSu) #update user embeddings
    theta_v = theta_v + (a*sumGv + a*sumSv) #update item embeddings
    new_eife = compute_e_ife(theta_u,theta_v,G_mini_items,S_items) #check S: expand S or use new mini S?
    difference = new_eife - e_ife
    if e_ife > new_eife:
        theta_u = theta_u - (a*sumGu + a*sumSu) #undo
        theta_v = theta_v - (a*sumGv + a*sumSv) #undo
    else:
        e_ife = new_eife
    return (difference,e_ife)
    
def implicit_feedback_embedding(G,b,a,k,row_length,col_length):
    #initiliase list of vector embeddings randomly
    theta_u = list()
    theta_v = list()
    for i in range(row_length+1):
        theta_u.append(np.random.rand(k))
    for i in range(col_length+1):
        theta_v.append(np.random.rand(k))   
    tolerance = 5 #convergence parameter epsilon
    convergence_difference = 100 #arbitrary to initialisation
    G_items = set(zip(G.row,G.col)) #get iterator for row,col pair with non-zero values
    e_ife = None
    G = G.tocsr()
    print(theta_u[0],theta_v[0])
    while abs(convergence_difference) > tolerance: #while embedding has not converged
        print("diff > tolerance?",abs(convergence_difference),tolerance)
        G_mini = mini_batch_generator(G,batch_size,row_length,col_length)
        S = negative_sampling(G_items,row_length,col_length,batch_size) #make negative feedback graph
        G_mini = G.tocoo()
        S_items = set(zip(S.row,S.col))
        G_mini_items = set(zip(G_mini.row,G_mini.col))
        if not e_ife:
            e_ife = compute_e_ife(theta_u,theta_v,G_mini_items,S_items) #initialise e_ife mini G or G?
        convergence_difference,e_ife = update_embeddings(theta_u,theta_v,G_mini_items,S_items,a,e_ife)
    print("diff > tolerance?",convergence_difference,tolerance)
    print(theta_u[0],theta_v[0])
     
#set parameters
batch_size = 2000
learning_rate = 0.00001
dimensionality = 5
#adjacency matrix representing the implicit Feedback Graph G
G = sparse.csr_matrix((data['click_count'].astype(float), (data['user_id'], data['item_id'])))#make user/item graph
G = G[:50000,:500000]
G = G.tocoo()
row_length = max(G.row)
col_length = max(G.col)
print("embedding...")
implicit_feedback_embedding(G,batch_size,learning_rate,dimensionality,row_length,col_length)
print("done")

embedding...
[0.00553571 0.95369939 0.00492435 0.63907834 0.37024874] [0.18337591 0.07101578 0.08701105 0.7087646  0.5545718 ]
diff > tolerance? 100 5
diff > tolerance? 7673.555257000466 5
diff > tolerance? 10.878830603269307 5
diff > tolerance? 13.797616818763345 5
diff > tolerance? 7.60986408525423 5
diff > tolerance? 20.447298009705264 5
diff > tolerance? 17.363680123507947 5
diff > tolerance? 13.531608344681445 5
diff > tolerance? 52.67578985281216 5
diff > tolerance? 13.79185055657581 5
diff > tolerance? 66.9639276594171 5
diff > tolerance? 65.7434700132253 5
diff > tolerance? 64.64932793752087 5
diff > tolerance? 86.51263550687509 5
diff > tolerance? 39.046483384652674 5
diff > tolerance? 46.75364152338079 5
diff > tolerance? 68.58276962830496 5
diff > tolerance? 25.33966212701489 5
diff > tolerance? 67.46510497642521 5
diff > tolerance? 54.469602206554555 5
diff > tolerance? 9.803119360571145 5
diff > tolerance? 14.805182739994052 5
diff > tolerance? 82.59660698894004 5
diff > t